<a href="https://colab.research.google.com/github/armandossrecife/my_validation/blob/main/avalia_modelo_inspecao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importa os dados para avaliação

In [60]:
!wget https://raw.githubusercontent.com/armandossrecife/my_validation/main/inspecao_merge_manual_chatgpt.xlsx

--2023-10-24 16:00:09--  https://raw.githubusercontent.com/armandossrecife/my_validation/main/inspecao_merge_manual_chatgpt.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67122 (66K) [application/octet-stream]
Saving to: ‘inspecao_merge_manual_chatgpt.xlsx’

inspecao_merge_manu 100%[===================>]  65.55K  --.-KB/s    in 0.07s   

2023-10-24 16:00:09 (914 KB/s) - ‘inspecao_merge_manual_chatgpt.xlsx’ saved [67122/67122]



# Funções de apoio


In [62]:
import pandas as pd
import collections
import matplotlib.pyplot as plt
import seaborn as sns
import random
import scipy.stats as stats
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

def convert_to_upper(list_of_strings):
  upper_list = []
  for string in list_of_strings:
    upper_list.append(string.upper())
  return upper_list

def conta_yes_no(my_list):
  conta_yes, conta_no = 0,0
  for each in my_list:
    if each == 'YES':
      conta_yes += 1
    else:
      conta_no += 1
  return conta_yes, conta_no

def convert_yes_no_to_int(dfa, dfb, my_column):
  labels = ["YES", "NO"]  # Define your possible labels here

  # Convert labels to numerical values
  label_to_num = {label: i for i, label in enumerate(labels)}
  dfa[my_column] = dfa[my_column].map(label_to_num)
  dfb[my_column] = dfb[my_column].map(label_to_num)
  return dfa, dfb

# Define a function to calculate Cohen's Kappa
def calculate_cohens_kappa(dfa, dfb, my_column):
    # Calculate Cohen's Kappa
    kappa = cohen_kappa_score(dfa[my_column], dfb[my_column])
    return kappa

def extra_metrics_yes(dfa, dfb, my_column):
  # Define your ground truth (actual values) and predicted values
  ground_truth = dfa[my_column]  # Ground truth values from manual inspection
  predicted = dfb[my_column]     # Predicted values from ChatGPT inspection

  # Calculate precision, recall, accuracy, and F1-score
  precision = precision_score(ground_truth, predicted, pos_label=1)
  recall = recall_score(ground_truth, predicted, pos_label=1)
  accuracy = accuracy_score(ground_truth, predicted)
  f1 = f1_score(ground_truth, predicted, pos_label=1)
  return precision, recall, accuracy, f1

# Importa dados originais

Merge da inspeção manual e inspeção via ChatGPt

In [63]:
df_inspection = pd.read_excel('inspecao_merge_manual_chatgpt.xlsx')
df_inspection.sample(5)

,issue_key,issue_type,summary,description,architecture_issue,i2_chatgpt_ai,TP,TN,FP,FN
104,CASSANDRA-16925,Improvement,Task Execution,To support CEP-10 it is necessary to support a...,Yes,Yes,1,0,0,0
56,CASSANDRA-5226,Improvement,CQL3 refactor to allow conversion function,"In CASSANDRA-5198, we've fixed CQL3 type valid...",Yes,Yes,1,0,0,0
22,CASSANDRA-8244,Bug,"Token, DecoratedKey, RowPosition and all bound...",Currently some of the functionality of Token r...,Yes,Yes,1,0,0,0
47,CASSANDRA-6230,Sub-task,Write hints to flat files instead of the syste...,Writing to a file would have less overhead on ...,Yes,Yes,1,0,0,0
68,CASSANDRA-3617,Improvement,Clean up and optimize Message,The Message class has grown largely by accreti...,No,No,0,1,0,0


In [64]:
df_inspection_check = df_inspection[['issue_key', 'issue_type', 'summary', 'architecture_issue', 'i2_chatgpt_ai', 'TP', 'TN', 'FP', 'FN']]
df_inspection_check.sample(10)

,issue_key,issue_type,summary,architecture_issue,i2_chatgpt_ai,TP,TN,FP,FN
50,CASSANDRA-5613,Improvement,Rename 'Table' class to 'Keyspace' in 2.0,Yes,Yes,1,0,0,0
98,CASSANDRA-17056,New Feature,CEP-17 – Pluggable SSTable format (SSTable for...,Yes,Yes,1,0,0,0
30,CASSANDRA-7563,Bug,"UserType, TupleType and collections in UDFs",No,No,0,1,0,0
106,CASSANDRA-16914,Improvement,Implement Virtual Tables for Auth Caches,Yes,Yes,1,0,0,0
136,CASSANDRA-14115,Sub-task,Refactor streaming,Yes,Yes,1,0,0,0
97,CASSANDRA-17147,New Feature,Guardrails prototype,Yes,Yes,1,0,0,0
55,CASSANDRA-5286,Improvement,Streaming 2.0,Yes,Yes,1,0,0,0
119,CASSANDRA-15299,Improvement,CASSANDRA-13304 follow-up: improve checksummin...,Yes,Yes,1,0,0,0
133,CASSANDRA-14227,Bug,Extend maximum expiration date,Yes,Yes,1,0,0,0
161,CASSANDRA-10091,New Feature,Integrated JMX authn & authz,Yes,Yes,1,0,0,0


# Dados da inspeção manual

In [65]:
df_inspection_check_manual = df_inspection_check[['issue_key', 'issue_type','summary','architecture_issue']]
df_inspection_check_manual['architecture_issue'] = df_inspection_check_manual['architecture_issue'].str.upper()
df_inspection_check_manual

<ipython-input-65-516850d7e04a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inspection_check_manual['architecture_issue'] = df_inspection_check_manual['architecture_issue'].str.upper()


,issue_key,issue_type,summary,architecture_issue
0,CASSANDRA-9975,Sub-task,Flatten Iterator call hierarchy with a shared ...,YES
1,CASSANDRA-994,Task,re-organize endpointsnitch implementations,YES
2,CASSANDRA-9932,Improvement,Make all partitions btree backed,YES
3,CASSANDRA-9921,Improvement,Combine MV schema definition with MV table def...,YES
4,CASSANDRA-9705,Sub-task,Simplify some of 8099's concrete implementations,YES
...,...,...,...,...
157,CASSANDRA-1072,Sub-task,Increment counters,YES
158,CASSANDRA-10661,Improvement,Integrate SASI to Cassandra,YES
159,CASSANDRA-1034,Bug,Remove assumption that Key to Token is one-to-one,YES
160,CASSANDRA-10109,Sub-task,Windows dtest 3.0: ttl_test.py failures,YES


In [66]:
df_inspection_check_manual.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   issue_key           162 non-null    object
 1   issue_type          162 non-null    object
 2   summary             162 non-null    object
 3   architecture_issue  162 non-null    object
dtypes: object(4)
memory usage: 5.2+ KB


In [67]:
ai_manual = df_inspection_check_manual.architecture_issue.to_list()
conta_yes, conta_no = conta_yes_no(my_list=ai_manual)
print(f'Soma Yes and No: {conta_yes+conta_no}, Yes: {conta_yes}, No: {conta_no}')
print(f'%Yes: {round((conta_yes/(conta_yes+conta_no)*100),2)}, %No: {round((conta_no/(conta_yes+conta_no)*100),2)}')

Soma Yes and No: 162, Yes: 136, No: 26
%Yes: 83.95, %No: 16.05


# Dados da inspeção via ChatGPT

In [68]:
df_inspection_check_chagpt = df_inspection_check[['issue_key', 'issue_type','summary','i2_chatgpt_ai']]
df_inspection_check_chagpt['architecture_issue'] = df_inspection_check_chagpt['i2_chatgpt_ai']
df_inspection_check_chagpt['architecture_issue'] = df_inspection_check_chagpt['architecture_issue'].str.upper()
df_inspection_check_chagpt = df_inspection_check_chagpt[['issue_key', 'issue_type', 'summary', 'architecture_issue']]
df_inspection_check_chagpt.sample(10)

<ipython-input-68-d2a9498a21d6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inspection_check_chagpt['architecture_issue'] = df_inspection_check_chagpt['i2_chatgpt_ai']
<ipython-input-68-d2a9498a21d6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inspection_check_chagpt['architecture_issue'] = df_inspection_check_chagpt['architecture_issue'].str.upper()


,issue_key,issue_type,summary,architecture_issue
117,CASSANDRA-15861,Bug,Mutating sstable component may race with entir...,NO
9,CASSANDRA-9431,Improvement,Static Analysis to warn on unsafe use of Autoc...,YES
125,CASSANDRA-14821,Improvement,Make it possible to run multi-node coordinator...,YES
80,CASSANDRA-18592,New Feature,CIDR filtering authorizer for Cassandra,YES
48,CASSANDRA-620,New Feature,Add per-keyspace replication factor (possibly ...,YES
76,CASSANDRA-2319,Improvement,Promote row index,NO
130,CASSANDRA-14436,Improvement,Add sampler for query time and expose with nod...,NO
16,CASSANDRA-8877,New Feature,Ability to read the TTL and WRITE TIME of an e...,NO
56,CASSANDRA-5226,Improvement,CQL3 refactor to allow conversion function,YES
114,CASSANDRA-16180,Task,4.0 Quality: Coordination Test Audit,NO


In [69]:
df_inspection_check_chagpt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   issue_key           162 non-null    object
 1   issue_type          162 non-null    object
 2   summary             162 non-null    object
 3   architecture_issue  162 non-null    object
dtypes: object(4)
memory usage: 5.2+ KB


In [70]:
ai_chatgpt = df_inspection_check_chagpt.architecture_issue.to_list()
conta_yes, conta_no = conta_yes_no(my_list=ai_chatgpt)
print(f'Soma Yes and No: {conta_yes+conta_no}, Yes: {conta_yes}, No: {conta_no}')
print(f'%Yes: {round((conta_yes/(conta_yes+conta_no)*100),2)}, %No: {round((conta_no/(conta_yes+conta_no)*100),2)}')

Soma Yes and No: 162, Yes: 123, No: 39
%Yes: 75.93, %No: 24.07


# Cálculo do Kappa


In [71]:
# Converte a coluna architecture_issue para 0 (NO) ou 1 (YES)
dfa, dfb = convert_yes_no_to_int(dfa=df_inspection_check_manual, dfb=df_inspection_check_chagpt, my_column='architecture_issue')

<ipython-input-62-fb1b080fa0ff>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfa[my_column] = dfa[my_column].map(label_to_num)


In [72]:
kappa_manual_chatgpt = calculate_cohens_kappa(dfa=df_inspection_check_manual, dfb=df_inspection_check_chagpt, my_column='architecture_issue')
print(round(kappa_manual_chatgpt, 3))

0.752


# Cálculo da performance do modelo de inspeção ChatGPT

In [73]:
my_precision, my_recall, my_accuracy, my_f1_score = extra_metrics_yes(dfa=df_inspection_check_manual, dfb=df_inspection_check_chagpt, my_column='architecture_issue')
print(f'my_precision: {round(my_precision, 3)}, my_recall: {round(my_recall, 3)}, my_accuracy: {round(my_accuracy, 3)}, my_f1_score: {round(my_f1_score,3)}')

my_precision: 0.667, my_recall: 1.0, my_accuracy: 0.92, my_f1_score: 0.8
